In [1]:
import numpy as np

#Forward mode, automatic differentiation class

##############################################
######       Arithmetic Operators       ######
##############################################
class forwardAD:
    def __init__(self, val, der_vector = None, numvar = 1, idx = 0):
        self.val = val
        if der_vector is None:
            der_vector = np.zeros(int(numvar))
            der_vector[idx] = 1
            self.der = der_vector
        else:
            self.der = der_vector
        #self.seed = np.floor(np.rand(1)*100000)

    #Overloaded addition
    def __add__(self, other):

        #For adding of the same class, add derivatives
        try:
            out_val = self.val + other.val
            out_der = self.der + other.der

        #For adding a float, keep derivative
        except AttributeError:
            out_val = self.val + other
            out_der = self.der

        return forwardAD(out_val, out_der)

    #Overloaded subtraction
    def __sub__(self, other):

        #For subtracting of the same class, subtract derivatives
        try:
            out_val = self.val - other.val
            out_der = self.der - other.der

        #For subtracting a float, keep derivative
        except AttributeError:
            out_val = self.val - other
            out_der = self.der

        return forwardAD(out_val, out_der)

    #Overloaded multiplication
    def __mul__(self, other):

        #For multiplying the same class, use product rule
        try:
            out_val = self.val * other.val
            out_der = self.der * other.val + self.val * other.der

        #For multiplying a float, multiply the value
        except AttributeError:
            out_val = self.val * other
            out_der = self.der * other

        return forwardAD(out_val, out_der)


    #Overloaded division
    def __truediv__(self, other):

        #For dividing the same class, use quotient rule
        try:
            out_val = self.val / other.val
            out_der = (self.der * other.val - self.val * other.der)/(other.val ** 2.0)

        #For dividing a float, divide the value
        except AttributeError:
            out_val = self.val / other
            out_der = self.der / other

        return forwardAD(out_val, out_der)
    
    def __truediv__(self, other):

        #For dividing the same class, use quotient rule
        try:
            out_val = self.val / other.val
            out_der = (self.der * other.val - self.val * other.der)/(other.val ** 2.0)

        #For dividing a float, divide the value
        except AttributeError:
            out_val = self.val / other
            out_der = self.der / other

        return forwardAD(out_val, out_der)

    #Overloaded power
    def __pow__(self, other):

    #For power to same class, use logarithmic differentiation
        try:
            out_val = self.val ** other.val
            out_der = (other.der * np.log(self.val) + other.val * (self.der / self.val)) * (self.val ** other.val)

        #For raising to a float, do power rule
        except AttributeError:
            out_val = self.val ** other
            out_der = self.der * other * self.val **(other - 1.0)

        return forwardAD(out_val, out_der)

    ### Reversed Operations ###

    # Addition is commutative
    def __radd__(self, other):
        return self.__add__(other)

    # Reverse subtraction
    def __rsub__(self, other):

        #If another of same type, then reverse order of subtraction
        try:
            out_val = other.val - self.val
            out_der = other.der - self.der

        #If a float, then reverse order subtract
        except AttributeError:
            out_val = other - self.val
            out_der = -1.0 * self.der

        return forwardAD(out_val, out_der)

    #Multiplication is commutative
    def __rmul__(self, other):
        return self.__mul__(other)

    #Reverse division
    def __rtruediv__(self, other):
        #For dividing the same class, use reversed quotient rule
        try:
            out_val = other.val / self.val
            out_der = (other.der * self.val - other.val * self.der)/(self.val ** 2.0)

        #For dividing a float, take derivative of f**(-1) using power rule
        except AttributeError:
            out_val = other / self.val
            out_der = (-1.0) * other * self.der / (self.val**2.0)

        return forwardAD(out_val, out_der)

    #Reverse power
    def __rpow__(self, other):

        #If base is same class, do logarithmic differentiation
        try:
            out_val = other.val ** self.val
            out_der = (self.der * np.log(other.val) + self.val * (other.der / other.val)) * (other.val ** self.val)

        #If base is a float, then b^x derivative rule
        except AttributeError:
            out_val = other ** self.val
            out_der = other ** self.val * self.der * np.log(other)

        return forwardAD(out_val, out_der)
    
    ##############################################
    ######       Comparison Operators       ######
    ##############################################
    
    def __lt__(self, other):
        try:
            return np.less(self.val, other.val) and np.less(self.der, other.der)
        except AttributeError:
            return np.less(self.val, other)
    
    def __gt__(self, other):
        try:
            return np.greater(self.val, other.val) and np.greater(self.der, other.der)
        except AttributeError:
            return np.greater(self.val, other)
            
    def __le__(self, other):
        try:
            return np.less_equal(self.val, other.val) and np.less_equal(self.der, other.der)
        except AttributeError:
            return np.less_equal(self.val, other)

    def __ge__(self, other):
        try:
            return np.greater_equal(self.val, other.val) and np.greater_equal(self.der, other.der)
        except AttributeError:
            return np.greater_equal(self.val, other)

    def __eq__(self, other):
        try:
            return np.array_equal(self.val, other.val) and np.array_equal(self.der, other.der)
        except AttributeError:
            return np.array_equal(self.val, other)
            
    #def __req__(self, other):
    #    return self.__eq__(other)
    
    def __ne__(self, other):
        try:
            return not np.array_equal(self.val, other.val) and not np.array_equal(self.der, other.der)
        except AttributeError:
            return not np.array_equal(self.val, other)
            
    #def __rne__(self, other):
    #    return self.__ne__(other)


##############################################
######       Elementary Functions       ######
##############################################

#For getting an 'e'
class e(forwardAD):
    def __init__(self, val=np.exp(1), der=0.0):
        self.val = val
        self.der = der

#For getting a log, defaults to natural log
class log(forwardAD):
    def __init__(self, inputs, base = np.exp(1)):

        #For getting log of variable expression, log is derivative/val
        try:
            self.val = np.log(inputs.val)/np.log(base)
            self.der = inputs.der / inputs.val

        #For log of float, use numpy.log
        except AttributeError:
            self.val = np.log(inputs) / np.log(base)
            self.der = 0.0


#####  Trignometric Functions #####

#For getting a sine
class sin(forwardAD):
    def __init__(self, inputs):

        #For getting sine of variable expression, der = in.der * cos(in.val)
        try:
            self.val = np.sin(inputs.val)
            self.der = inputs.der * np.cos(inputs.val)

        #For sine of float, use numpy.sin
        except AttributeError:
            self.val = np.sin(inputs)
            self.der = 0.0


#For getting a cosine
class cos(forwardAD):
    def __init__(self, inputs):

        #For getting cosine of variable expression, der = -in.der * sin(in.val)
        try:
            self.val = np.cos(inputs.val)
            self.der = -inputs.der * np.sin(inputs.val)

        #For cosine of float, use numpy.cos
        except AttributeError:
            self.val = np.cos(inputs)
            self.der = 0.0


#For getting a tangent
class tan(forwardAD):
    def __init__(self, inputs):

        #For getting tangent of variable expression, der = in.der * (1 + tan(in.val)^2)
        try:
            self.val = np.tan(inputs.val)
            self.der = inputs.der * (1 + np.tan(inputs.val)**2)

        #For tanget of float, use numpy.tan
        except AttributeError:
            self.val = np.tan(inputs)
            self.der = 0.0


##### Inverse Trignometric Functions #####

#For getting a inverse sine
class arcsin(forwardAD):
    def __init__(self, inputs):

        #For getting arcsin of variable expression, der = in.der * (1/sqrt(1-inputs.val**2))
        try:
            self.val = np.arcsin(inputs.val)
            self.der = inputs.der * (1/np.sqrt(1-inputs.val**2))

        #For arcsin of float, use numpy.arcsin
        except AttributeError:
            self.val = np.arcsin(inputs)
            self.der = 0.0


#For getting a inverse cosine
class arccos(forwardAD):
    def __init__(self, inputs):

        #For getting arccos of variable expression, der = -in.der * sin(in.val)
        try:
            self.val = np.arccos(inputs.val)
            self.der = -inputs.der * (1/np.sqrt(1-inputs.val**2))

        #For arccos of float, use numpy.arccos
        except AttributeError:
            self.val = np.arccos(inputs)
            self.der = 0.0


#For getting a inverse tangent
class arctan(forwardAD):
    def __init__(self, inputs):

        #For getting arctan of variable expression, der = in.der * (1 / in.val^2)
        try:
            self.val = np.arctan(inputs.val)
            self.der = inputs.der * (1/(1+inputs.val**2))

        #For arctan of float, use numpy.arctan
        except AttributeError:
            self.val = np.arctan(inputs)
            self.der = 0.0


##### Hyperbolic Trignometric Functions #####

#For getting a hyperbolic sine
class sinh(forwardAD):
    def __init__(self, inputs):

        #For getting sinh of variable expression, der = in.der * sinh(in.val)
        try:
            self.val = np.sinh(inputs.val)
            self.der = np.cosh(inputs.val)

        #For sine of float, use numpy.tan
        except AttributeError:
            self.val = np.sinh(inputs)
            self.der = 0.0


#For getting a hyperbolic cosh
class cosh(forwardAD):
    def __init__(self, inputs):

        #For getting cosh of variable expression, der = in.der * sinh(in.val)
        try:
            self.val = np.cosh(inputs.val)
            self.der = np.sinh(inputs.val)

        #For sine of float, use numpy.cos
        except AttributeError:
            self.val = np.cosh(inputs)
            self.der = 0.0

#For getting a hyperbolic tangent
class tanh(forwardAD):
    def __init__(self, inputs):

        #For getting tangent of variable expression, der = in.der * (1 + tan(in.val)^2)
        try:
            self.val = np.tanh(inputs.val)
            self.der = inputs.der * (1 - np.tanh(inputs.val)**2)

        #For sine of float, use numpy.tan
        except AttributeError:
            self.val = np.tanh(inputs)
            self.der = 0.0
class vector_func: #vector_func(f1, f2, ...)
    def __init__(self, *args): #  
        jacobian_array = []
        values_array = []
        for function in args: 
                jacobian_array.append(function.der)
                values_array.append(function.val)
        self.jacobian_array = np.array(jacobian_array)
        self.values_array = np.array(values_array)

    def jacobian(self):
        return self.jacobian_array
    def values(self):
        return self.values_array

    
        

In [2]:
cur_x = forwardAD(3) # The algorithm starts at x=3

rate = 0.01 # Learning rate
precision = 0.000001 #This tells us when to stop the algorithm
previous_step_size = 1 #
max_iters = 10000 # maximum number of iterations
iters = 0 #iteration counter

In [3]:
while previous_step_size > precision and iters < max_iters:
    prev_x = cur_x #Store current x value in prev_x
    f = (prev_x + 5)**2 # gradient
    cur_x = cur_x - rate * f.der #Grad descent
    previous_step_size = abs(cur_x.val - prev_x.val) #Change in x
    iters += 1 #iteration count
    print("Iteration",iters,"\nX value is", cur_x.val) #Print iterations
    
print("The local minimum occurs at", cur_x.val)

Iteration 1 
X value is [2.84]
Iteration 2 
X value is [2.6832]
Iteration 3 
X value is [2.529536]
Iteration 4 
X value is [2.37894528]
Iteration 5 
X value is [2.23136637]
Iteration 6 
X value is [2.08673905]
Iteration 7 
X value is [1.94500427]
Iteration 8 
X value is [1.80610418]
Iteration 9 
X value is [1.6699821]
Iteration 10 
X value is [1.53658246]
Iteration 11 
X value is [1.40585081]
Iteration 12 
X value is [1.27773379]
Iteration 13 
X value is [1.15217911]
Iteration 14 
X value is [1.02913553]
Iteration 15 
X value is [0.90855282]
Iteration 16 
X value is [0.79038176]
Iteration 17 
X value is [0.67457413]
Iteration 18 
X value is [0.56108265]
Iteration 19 
X value is [0.44986099]
Iteration 20 
X value is [0.34086377]
Iteration 21 
X value is [0.2340465]
Iteration 22 
X value is [0.12936557]
Iteration 23 
X value is [0.02677826]
Iteration 24 
X value is [-0.07375731]
Iteration 25 
X value is [-0.17228216]
Iteration 26 
X value is [-0.26883652]
Iteration 27 
X value is [-0.363

Iteration 370 
X value is [-4.99546332]
Iteration 371 
X value is [-4.99555406]
Iteration 372 
X value is [-4.99564297]
Iteration 373 
X value is [-4.99573011]
Iteration 374 
X value is [-4.99581551]
Iteration 375 
X value is [-4.9958992]
Iteration 376 
X value is [-4.99598122]
Iteration 377 
X value is [-4.99606159]
Iteration 378 
X value is [-4.99614036]
Iteration 379 
X value is [-4.99621755]
Iteration 380 
X value is [-4.9962932]
Iteration 381 
X value is [-4.99636734]
Iteration 382 
X value is [-4.99643999]
Iteration 383 
X value is [-4.99651119]
Iteration 384 
X value is [-4.99658097]
Iteration 385 
X value is [-4.99664935]
Iteration 386 
X value is [-4.99671636]
Iteration 387 
X value is [-4.99678204]
Iteration 388 
X value is [-4.99684639]
Iteration 389 
X value is [-4.99690947]
Iteration 390 
X value is [-4.99697128]
Iteration 391 
X value is [-4.99703185]
Iteration 392 
X value is [-4.99709121]
Iteration 393 
X value is [-4.99714939]
Iteration 394 
X value is [-4.9972064]
Ite

In [4]:
def gradient_descent(variables, f, cur_x, rate, precision, previous_step_size, max_iters):
    if isinstance(f, str):

    #If just one string, convert to list
        if isinstance(variables, str):

            variables = [variables]

        #Convert to numpy arrays
        variables = np.array(variables)
        cur_x = np.array(cur_x)

        iters = 0
        while previous_step_size > precision and iters < max_iters:
            for index, variable in enumerate(variables):

                #Store to variable name
                name = variable

                #Save as forwardAD object
                exec(name + " = forwardAD(np.array(cur_x), numvar = len(variables), idx = index)")

            #Stores full function as a forwardAD object
            AD_function = eval(f)
            
            prev_x = cur_x #Store current x value in prev_x
            cur_x = cur_x - rate * AD_function.der #Grad descent
            previous_step_size = abs(cur_x - prev_x) #Change in x
            iters += 1 #iteration count
            print("Iteration",iters,"\nX value is", cur_x) #Print iterations

    print("The local minimum occurs at", cur_x)

In [5]:
variables = "x"
f = "(x+5)**2"
cur_x = [3]
rate = 0.01
precision = 0.000001
previous_step_size = 1
max_iters = 10000
iters = 0
gradient_descent(variables, f, cur_x, rate, precision, previous_step_size, max_iters)

Iteration 1 
X value is [2.84]
Iteration 2 
X value is [2.6832]
Iteration 3 
X value is [2.529536]
Iteration 4 
X value is [2.37894528]
Iteration 5 
X value is [2.23136637]
Iteration 6 
X value is [2.08673905]
Iteration 7 
X value is [1.94500427]
Iteration 8 
X value is [1.80610418]
Iteration 9 
X value is [1.6699821]
Iteration 10 
X value is [1.53658246]
Iteration 11 
X value is [1.40585081]
Iteration 12 
X value is [1.27773379]
Iteration 13 
X value is [1.15217911]
Iteration 14 
X value is [1.02913553]
Iteration 15 
X value is [0.90855282]
Iteration 16 
X value is [0.79038176]
Iteration 17 
X value is [0.67457413]
Iteration 18 
X value is [0.56108265]
Iteration 19 
X value is [0.44986099]
Iteration 20 
X value is [0.34086377]
Iteration 21 
X value is [0.2340465]
Iteration 22 
X value is [0.12936557]
Iteration 23 
X value is [0.02677826]
Iteration 24 
X value is [-0.07375731]
Iteration 25 
X value is [-0.17228216]
Iteration 26 
X value is [-0.26883652]
Iteration 27 
X value is [-0.363

X value is [-4.96647595]
Iteration 272 
X value is [-4.96714643]
Iteration 273 
X value is [-4.9678035]
Iteration 274 
X value is [-4.96844743]
Iteration 275 
X value is [-4.96907848]
Iteration 276 
X value is [-4.96969691]
Iteration 277 
X value is [-4.97030297]
Iteration 278 
X value is [-4.97089691]
Iteration 279 
X value is [-4.97147898]
Iteration 280 
X value is [-4.9720494]
Iteration 281 
X value is [-4.97260841]
Iteration 282 
X value is [-4.97315624]
Iteration 283 
X value is [-4.97369312]
Iteration 284 
X value is [-4.97421925]
Iteration 285 
X value is [-4.97473487]
Iteration 286 
X value is [-4.97524017]
Iteration 287 
X value is [-4.97573537]
Iteration 288 
X value is [-4.97622066]
Iteration 289 
X value is [-4.97669625]
Iteration 290 
X value is [-4.97716232]
Iteration 291 
X value is [-4.97761908]
Iteration 292 
X value is [-4.97806669]
Iteration 293 
X value is [-4.97850536]
Iteration 294 
X value is [-4.97893525]
Iteration 295 
X value is [-4.97935655]
Iteration 296 
X 

X value is [-4.99990035]
Iteration 560 
X value is [-4.99990235]
Iteration 561 
X value is [-4.9999043]
Iteration 562 
X value is [-4.99990621]
Iteration 563 
X value is [-4.99990809]
Iteration 564 
X value is [-4.99990993]
Iteration 565 
X value is [-4.99991173]
Iteration 566 
X value is [-4.99991349]
Iteration 567 
X value is [-4.99991522]
Iteration 568 
X value is [-4.99991692]
Iteration 569 
X value is [-4.99991858]
Iteration 570 
X value is [-4.99992021]
Iteration 571 
X value is [-4.9999218]
Iteration 572 
X value is [-4.99992337]
Iteration 573 
X value is [-4.9999249]
Iteration 574 
X value is [-4.9999264]
Iteration 575 
X value is [-4.99992788]
Iteration 576 
X value is [-4.99992932]
Iteration 577 
X value is [-4.99993073]
Iteration 578 
X value is [-4.99993212]
Iteration 579 
X value is [-4.99993347]
Iteration 580 
X value is [-4.99993481]
Iteration 581 
X value is [-4.99993611]
Iteration 582 
X value is [-4.99993739]
Iteration 583 
X value is [-4.99993864]
Iteration 584 
X va